# Analysis
**Note:** Ensure you have completed the `Setup.ipynb` notebook.

Now is also a good time to look through `schema.py` and familiarize yourself with the fields on each
DB model (if you're so inclined, although this notebook will hold your hand). You might also find
knowledge of the [Peewee](http://docs.peewee-orm.com) library for Python helpful.

As before, replace the `folder_name` below with one you've already set up.

In [ ]:
# NOTE: Please change the folder name in `constants.py`.
from constants import folder_name

In [ ]:
from schema import get_chat_db

ChatDB = get_chat_db(folder_name)
ChatDB.open()

## Most Common Words
A natural starting place for analysis is figuring out the most common words in our chat (excluding
particles like `the` and `a`). We also would like to see the breakdown of common words by
individual.

First, let's use `nltk` to get a list of particle words (known more commonly as _stopwords_) for our
chat language (which I'm assuming is English here, but please change it as you need to).

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

Let's also build an auxiliary table around word usage (based on the full-text search index for
messages):

In [ ]:
MessageIndex = ChatDB.MessageIndex
# Peewee has functionality for VocabModel, but for some reason does not actually run a CREATE.
MessageIndex.raw(
    "CREATE VIRTUAL TABLE IF NOT EXISTS messageindex_v USING fts5vocab(messageindex, instance);"
).execute()
MessageVocab = MessageIndex.VocabModel(table_type="instance")
MessageVocab._meta.remove_field("cnt")  # Not present for VocabModel type `instance`.

If that didn't make a ton of sense, and you're still curious, just know the following:
- The `Message` table is referenced by the `MessageIndex` such that
  `Message.id == MessageIndex.rowid` and `Message.content == MessageIndex.content` (the content is
  duplicated, but `MessageIndex` uses efficient text indexing to allow full-text querying over its
  `content` field).
- `MessageVocab` is a table consisting of columns `term` and `doc` (and other stuff, but it's not
  super relevant to us). For every occurrence of a word in a particular `MessageIndex` entry, there
  is an entry in `MessageVocab` where `term` equals the word and `doc` equals the `rowid` from
  `MessageIndex`.
    - If we had values `banana tomato` and `carrot tomato` as the entries in `MessageIndex`, then
      `MessageVocab` would have entries `banana, 0`, `tomato, 0`, `tomato, 1`, `carrot, 1`.

**TLDR:** Now let's figure out the top 10 words in our chat. 

In [ ]:
from peewee import fn, SQL

def print_top_ten(results, filter_fn=None):
    # Collect results and filter stopwords.
    counts = [(result.term, result.count) for result in results]
    counts = list(filter(lambda x: x[0] not in stop_words, counts))

    if filter_fn is not None:
        counts = list(filter(filter_fn, counts))
    
    # Print top 10 results.
    for word, count in counts[:10]:
        print(f"{word} ({count})")
        
# Build query (words ordered by count, descending).
results = (
    MessageVocab.select(MessageVocab.term, fn.COUNT(MessageVocab.rowid).alias("count"))
    .group_by(MessageVocab.term)
    .order_by(SQL("count").desc())
)

print_top_ten(results)

That's pretty interesting, but if your chat is anything like the ones I tested this on, you still
might have some tiny filler words. Let's try that again, but limit words to 5 characters or more.

In [ ]:
print_top_ten(results, filter_fn=lambda x: len(x[0]) > 4)

And now let's print the top ten non-stopwords by user:

In [ ]:
Message = ChatDB.Message
users = ChatDB.User.select()

for user in users:
    results = (
        MessageVocab.select(MessageVocab.term, fn.COUNT(MessageVocab.rowid).alias("count"))
        .join(Message, on=(Message.id == MessageVocab.doc))
        .where(Message.sender == user)
        .group_by(MessageVocab.term)
        .order_by(SQL("count").desc())
    )
    
    print(f"User: {user}")
    print_top_ten(results)
    print()

**Surprised?** Don't remember using `lmao` or whatever so many times? Use the function below to list
the messages containing a particular word...

In [ ]:
def find_messages_with(word):
    results = (
        Message.select(Message.id, Message.sender, Message.content)
        .distinct()  # Note the important thing: this is distinct on id.
        .join(MessageVocab, on=(Message.id == MessageVocab.doc))
        .where(MessageVocab.term == word)
    )

    return [(result.sender.name, result.content) for result in results]

In [ ]:
find_messages_with('zuckerberg')